## Топографическая сегментация ##

### Архитектура ###

Так как мы имеем дело с сегментацией изображений средних размеров на достаточно неплохо визуально отделимые и относительно гомогенные классы, есть надежда, что с этим справится UNet небольшого размера. Известно, что UNet достаточно хорошо тренируется на малых датасетах. 

Будем тренировать UNet из 4-х пар блоков. С 5 парами количество параметров выходит за пределы вычислительных возможностей моего компьютера. 
Мой конфиг: 
- i5-7300 4@2.5Ghz
- 8GB RAM
- GTX1050 2Gb
- HDD

Исходные изображения нарежем на произвольные (почти) квадратные куски и отмасштабируем до размера 128*128. Выбор размера обучающей картинки обусловлен следующим. Для корректной работы используемой реализации UNet требуются входные данные со сторонами, кратными $2^\text{количество пар блоков}$ (иначе pooling-слои приводят к тому, что выходное разрешение меньше входного). Далее экспериментально подобран размер, обеспечивающий хоть какой-то batching при тренировке.

В качестве дополнительных аугментаций применяется случайное отражение по обоим осям. Вращения не применяются, так как реально они не увеличивают разнообразие данных, в то же время лишь добавляют артефактов. Шумы и цветовые смещения также не применяются ввиду относительной однородности тренировочных и валидационных данных.

Данные загружаются с диска один раз и в дальнейшем все манипуляции происходят в памяти, чтобы снизить нагрузку на HDD у условиях небольшого набора данных для обучения. Аугментации применяются на лету по расписанию Dataloader'а, нарезанные изображения не хранятся.

### Инференс на исходных изображениях ###
С целью обеспечения работы модели на исходных изображений реализован тайлер на без пакета `pytorch_toolbelt`. При этом шаг замощения в 2 раза меньше размера плитки, предсказания
на перекрывающихся областях усредняются, паддинг осуществляется с отражением.

### Метрика и энергия ###
Так как для конкретных вырезанных кусочков может наблюдаться существенных перекос по присутствующим
классам, отличных от среднего распределения, вместо логарифмического подобия (`CrossEntropy / softmax+NLLLoss`) используется `FocalLoss`, который умеет адаптироваться к дисбалансу классов (на самом деле преобразование корня бывает полезным для выравнивания гистограмм в широчайшем круге задач).

Так как в задаче нет существенных прагматических ограничений, что могло бы потребовать приоритезации каких-то из стандартных отношений predicted/ground_truth, будем использовать попиксельный `F1_score`. Технически реализована возможность передавать веса для усреднения по классам, но реально я всегда выславляю их 1:1:1

### Обучение ###
Модель показала приемлемые результаты после 1 часа тренировки (120 000 изображений), поэтому  не стал далее экспериментировать. Интересно, что уже после тренировки на 4000 изображений, на что уходит пара минут, модель выдает уже вполне разумные результаты.

Используется оптимизатор ADAM с `lr=0.01` и параметрами по умолчанию в паре с экспоненциальным планировщиком скорости обучения. Коэффициент затухания `0.9999` на эпоху, эпоха состоит из 1 батча в 4 изображения (по одному фрагменту из каждого из исходных изображений).

Вычисленные веса приложены, весь код тренировки сохранен.

### Постпроцессинг ###
Изучение результатов работы сети показало, что она распознает кассы *слишком* точно, выделяя опушки леса как "прочее", а лесополосы и деревья на приусадебных участках как "лес". В условиях ограниченного времени я ограничился только двумя фильтрами на базе `skimage`: замыкания (композиция раздутия и эрозии) и заполнения пустот по пороговому значению площади. Коэффициенты подобраны эмпирически (фильтры работают на cpu и небыстро, поэтому запускать рандомизированный gridsearch не было времени; да и тенденция по среднему изменению F1 была достаточно явной).

In [1]:
import data1
import torch, torch.utils, torch.utils.data
from unet import UNet
from tqdm.notebook import tqdm as tqdm
from torchsummary import summary
import modeltools1
import datetime
import torchvision
import os
import filters1
from itertools import product

device = torch.device("cuda")

#image output folder
out_dir = "out1"
if not os.path.exists(out_dir): os.mkdir(out_dir)

#weights trained on 4 * 30000 image fragments, set to None to force retraining
weights_path = "weights-FocalLoss-30000epochs.dat"
#weights_path = None

#paths to datasets
train_data_dir = "../01_image_segmentation1/01_train"
train_data_filename = "idx-train.txt"
val_data_dir = "../01_image_segmentation1/02_test_clean"
val_data_filename = "idx-test.txt"


In [2]:
# train dataset. Upon creation dataset[ind] produces 128*128 images, arbitrarily
# chosen from idx'th input image. Thus dataset[0] gives different results it is
# called. Accepts an optional size output_size parameter defaulted to (128,128) 
dataset_train = data1.Dataset(train_data_dir, train_data_filename)

dataset_val = data1.Dataset(val_data_dir, val_data_filename)
# this method switches a dataset to a raw mode, i.e. after dataset.eval()
# dataset[idx] stably produces the idx'th original image
dataset_val.eval()

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, num_workers=1, 
    pin_memory=True, persistent_workers=True)
dataloader_val = torch.utils.data.DataLoader(dataset_train, batch_size = 4, num_workers=1, 
    pin_memory=True, persistent_workers=True)

Loading raw data: 0it [00:00, ?it/s]

Loading raw data: 0it [00:00, ?it/s]

In [3]:
model = UNet(in_channels = 3, out_channels=3, n_blocks = 4).to(device)
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
              ReLU-2         [-1, 32, 128, 128]               0
       BatchNorm2d-3         [-1, 32, 128, 128]              64
            Conv2d-4         [-1, 32, 128, 128]           9,248
              ReLU-5         [-1, 32, 128, 128]               0
       BatchNorm2d-6         [-1, 32, 128, 128]              64
         MaxPool2d-7           [-1, 32, 64, 64]               0
         DownBlock-8  [[-1, 32, 64, 64], [-1, 32, 128, 128]]               0
            Conv2d-9           [-1, 64, 64, 64]          18,496
             ReLU-10           [-1, 64, 64, 64]               0
      BatchNorm2d-11           [-1, 64, 64, 64]             128
           Conv2d-12           [-1, 64, 64, 64]          36,928
             ReLU-13           [-1, 64, 64, 64]               0
      BatchNorm2d-14      

In [4]:
lr = 0.01
optim_params = model.parameters()
optim = torch.optim.Adam(optim_params, lr=lr)
# loss = torch.nn.CrossEntropyLoss()
loss = modeltools1.FocalLoss(reduction="mean")
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma =0.9999)
epochs = 1000 #set to at least 30 000 to reproduce the pretrained weights performance

In [5]:
# the model is loaded if path to weights is specified, otherwise trains itself

losses = []
if ("weights_path" in locals() or "weights_path" in globals()) and weights_path:
    model.load_state_dict(torch.load(weights_path))
else:
    modeltools1.train_model(model, loss, optim, scheduler, dataloader_train,
        num_epochs = epochs, device = device) #30 000 takes one hour and suffices
    model.cpu()
    current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    torch.save(model.state_dict(), './weights'+current_time+".dat")

Проверим на одном изображении.

Я провожу большинство проверок на train'е, так как на момент вечера воскресенья, валидационный датасет все еще был битый (маски не совпадают по размеру с исходным изораженим).

In [7]:
dataset_train.eval()
image = dataset_train[0][0]
label = dataset_train[0][1]
label_masks = modeltools1.classes_to_masks(label)

model.cuda()

prediction = modeltools1.tiled_eval(model, image, 16, (128,128), (64,64), device, device)
pred_mask = modeltools1.logits_to_masks(prediction)
masked = modeltools1.apply_mask(image, pred_mask, colors=modeltools1.DEFAULT_OVERLAY_COLORS, alpha=0.3)
torchvision.io.write_png(masked, out_dir+"/test.png")
score = modeltools1.F1_score(pred_mask, label_masks)
print("Score: ", score)

  0%|          | 0/44 [00:00<?, ?it/s]

Score:  0.8979603275315912


Ну и посчитаем срденее по трейнсету. При этом в папке `out1` появятся: исходные изображения, сгенерированные и эталонные маски, оверлей обоих масок на исходное изображение.

In [8]:

dataset_train.eval()
total_score = modeltools1.eval_on_dataset(model, dataset_train, out_dir+"/train", 8,
    (128,128), (64,64), device, device)

print(f"Average F1 score on train set: {total_score}")

Images in dataset:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

Score 0: 0.8979603275315912


  0%|          | 0/51 [00:00<?, ?it/s]

Score 1: 0.9183278604105028


  0%|          | 0/30 [00:00<?, ?it/s]

Score 2: 0.9096528753406101


  0%|          | 0/102 [00:00<?, ?it/s]

Score 3: 0.9245349943980967
Average F1 score on train set: 0.9126190144202002


`Average F1 score on train set: 0.9126190144202002` выглядит уже неплохо, но вот непосредственное изучение результата показывает проблему:
![](prob_demo1.png)

Сеть распознает классы слишком локально. Понятно, что бОльшую сеть, тренируемую на бОльших фрагментах можно от этого отучить, на текущую архитектуру тут надежды не очень много, поэтому отфильтруем эвристически. Сначала попробуем на одном изображении. Тут можно поподбирать параметры.

In [10]:
dataset_train.eval()
image, label = dataset_train[0]
prediction = modeltools1.tiled_eval(model, image, 16, (128,128), (64,64), device, device)
pred_mask = modeltools1.logits_to_masks(prediction)
label_mask = modeltools1.classes_to_masks(label)
score = modeltools1.F1_score(pred_mask, label_mask)

pred_fitered = filters1.remove_small_holes(pred_mask, 10, 200)
filtered_score = modeltools1.F1_score(pred_fitered, label_mask)

print(f"Original score: {score}, filtered score: {filtered_score}")

  0%|          | 0/44 [00:00<?, ?it/s]

Original score: 0.8979603275315912, filtered score: 0.9297780981072274


Прогресс налицо, поэтому прогоним на всех изображениях. Результат можно наблюдать в папке out. Здесь привожу один пример.
![](prob_demo2.png)

In [6]:
# Compute F1 with filtration on train dataset
dataset_train.eval()
total_score = modeltools1.eval_on_dataset(model, dataset_train, out_dir+"/train", 8,
    (128,128), (64,64), device, device, [], True, 10 ,300)

print(f"Average F1 score after filtering on train set: {total_score}")

Images in dataset:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

Score 0: 0.929835574907831


  0%|          | 0/51 [00:00<?, ?it/s]

Score 1: 0.930605983005312


  0%|          | 0/30 [00:00<?, ?it/s]

Score 2: 0.9362953003445535


  0%|          | 0/102 [00:00<?, ?it/s]

Score 3: 0.9479652156513637
Average F1 score after filtering on train set: 0.9361755184772651


Так же прогоним на валидационных данных, когда их починят....

In [7]:
total_score = modeltools1.eval_on_dataset(model, dataset_val, out_dir+"/val", 8,
    (128,128), (64,64), device, device, [], True, 10, 300)

print(f"Average F1 score on val set: {total_score}")

Images in dataset:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Score 0: 0.9006673085498438


  0%|          | 0/41 [00:00<?, ?it/s]

Score 1: 0.9524839651320565


  0%|          | 0/58 [00:00<?, ?it/s]

ValueError: Found input variables with inconsistent numbers of samples: [1979450, 1974875]

### Выводы ###

Видно, что модель дает хорошее начальное приближение для последующей обработки эвристическими методами. Предложенная фильтрация далека от идеала (на самом деле я попробовал первую пришедшую в голову идею и результат меня устроил). 

В последней момент мне пришла в голову идея вероятностного фильтра для того, чтобы сделать разметку более "полигональной", но реализовать я не успеваю. Вместо closing-фильтра сделать следующее: кидать в карту случайным образом маленькие треугольники. Скажем, со сторонами до 10 (20? 30?) пикселей. Если все три вершины одного цвета (на исходной карте без учета уже накиданных треугольников), то на новой карте весь треугольник красится в тот же цвет. Это должно и "замкнуть" малые "дыры" и сделать границы хотя бы ломанной. В случае наложения двух таких треугольников "побеждает" более приоритетный слой (дома над лесами, леса над дефолтом). А потом може еще линеаризовать границы так: запустить детектор границ (Лаплас?), а потом в найденные точки граници покидать отрезки. Дальше смотрим на k-пиксельную окретсность отрезка (k~3-10) и считаем в ней точки разных классов. Если соотношение примерно 1:1:0.01, значит эта прямая разделяет первые два класса два класса. Заливаем полоину ее окретсности в один цвет, вторую -- в другой и вокруг начала и конца чуть сглаживаем углы (гауссом?). 